# <font color='blue'> <center> Classic Control: Control theory problems from the classic RL literature </center> </font>

<br><br>

In this notebook we will present some classic environments in Reinforcement Learning research. These environments have continuous states spaces (i.e., infinite possible states) and therefore tabular methods cannot solve them. To tackle these environments (and more complex ones) we will have two tools:

- Extend the tabular methods with the techniques of discretization and tile coding
- Use function approximators (Neural Networks)

<br>

## <font color='#2874A6'> Tabla de Contenido </font>

1. [Modules](#1)
2. [Setup Code](#2)
3. [Algorithm](#3)
4. [Results](#4)

<a name="1"></a>
## <font color='#0E6655'> 1. Modules </font> 